<h1><center>Projet Integre</center></h1>

## <span style="color:yellow">Description des fichiers</span>

### publication (id_publication, date_pub, nbr_publication, article_title) 

author_publication (id_author, id_publication)  
author (id_author, name_author, nbr_publication)

keyword_publication (id_publication, keyword, nbr_use_keyword)  
keyword (keyword, nbr_used)

year_publication (id_publication, id_year)  
year (id_year, year)

publication_venue (id_publication, id_venue)  
venue (id_venue, name_venue)

## <span style="color:yellow">Data Cleaning (bibliothèques re, string), text preprocessing techniques</span>

  * suppression des colonnes inutiles
  *   mettre les titres en minuscule
  *   supprimer la ponctuation
  *   supprimer les valeurs numériques
  *   supprimer les élément qui n'ont aucune signification (balise et autre spéciaux)
  *   tokénisation (uni-gram d'abord)
  *   suppression des mots vides (stop words) 


###  Advanced data cleaning : 

  * racinisation (stemming) / lemmatisation 
  * tokénisation avancée : bi-grams ou tri-grams  
  * gestion des erreurs de frappe ? 



### Organisation des données (sklearn, countvectorizer)
   * corpus (ensemble des titres d'article), 
   * création de la matrice documents-termes (sklearn, countvectorizer)

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import gensim
import pickle


# librairies qui seront utiles ensuite
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import defaultdict
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords




# option d'affichage pour une meilleur visibilité
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#pd.set_option('display.min_rows', 25)
#pd.set_option('display.max_rows', 60)

In [2]:
import pickle

def save_dict(dico, filename) :
    with open("dict/"+filename+".pickle", 'wb') as handle:
        pickle.dump(dico, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_dict(filename) :
    with open("dict/"+filename+".pickle", 'rb') as handle:
        return(pickle.load(handle))

In [4]:
# Lecture des fichiers csv

author=pd.read_csv('data/author.csv',sep=',',encoding='cp1252') # (id_author, name_author)
publication=pd.read_csv('data/publication.csv',sep=',',encoding='cp1252') # Fichier des
del publication['categorie'] # suppression de la colonne qui ne contient que des proceeding
keyword=pd.read_csv('data/keyword.csv',sep=',',encoding='cp1252')
year=pd.read_csv('data/year.csv',sep=',',encoding='cp1252')


author_publication=pd.read_csv('data/publication_author.csv',sep=',',encoding='cp1252')
keyword_publication=pd.read_csv('data/Publication_keywords.csv',sep=',',encoding='cp1252')
year_publication=pd.read_csv('data/publication_year.csv',sep=',',encoding='cp1252')

In [5]:
keyword.shape[0] # 17585 mots clefs

17585

### <span style="color:Pink">Travail sur le fichier publication</span>

#### <span style="color:red">Suppression des doublons et préparation des données avant nettoyage = titre correct des publications</span>

In [6]:
publication.head() # Aperçu du fichier

,id_publication,date_pub,nbr_authors,article_title
0,conf/3dim/AbarghoueiB19,2019-11-05,2,To Complete or to Estimate. That is the Question: A Multi-Task Approach to Depth Completion and Monocular Depth Estimation.
1,conf/3dim/AbbeloosCCTD17,2018-07-03,5,3D Object Discovery and Modeling Using Single RGB-D Images Containing Multiple Object Instances.
2,conf/3dim/AbrevayaWB18,2018-10-26,3,Spatiotemporal Modeling for Efficient Registration of Dynamic 3D Faces.
3,conf/3dim/AdanR19,2019-11-05,2,Reconstruction of As-is Semantic 3D Models of Unorganised Storehouses.
4,conf/3dim/AhmedMG17,2019-09-25,3,Point Cloud Registration with Virtual Interest Points from Implicit Quadric Surface Intersections.


In [7]:
publication[publication.duplicated()] # Doublons sur les lignes

,id_publication,date_pub,nbr_authors,article_title
8205,conf/aabi/LueckmannBKM18,2019-07-12,4,Likelihood-free inference with emulator networks.
52438,conf/birthday/0001ZS18,2018-12-02,3,Network Security Metrics: From Known Vulnerabilities to Zero Day Attacks.
52441,conf/birthday/2017brinksma,2019-05-14,3,ModelEd. TestEd. TrustEd - Essays Dedicated to Ed Brinksma on the Occasion of His 60th Birthday
52443,conf/birthday/2017downey,2019-05-14,6,Computability and Complexity - Essays Dedicated to Rodney G. Downey on the Occasion of His 60th Birthday
52445,conf/birthday/2017larsen,2019-05-14,6,Models. Algorithms. Logics and Tools - Essays Dedicated to Kim Guldstrand Larsen on the Occasion of His 60th Birthday
141352,conf/eusipco/0042HH19,2019-11-25,3,Sensor Selection and Rate Distribution Based Beamforming in Wireless Acoustic Sensor Networks.
334051,conf/konvens/0002W16,2019-11-28,2,A Study on Gaps and Syntactic Boundaries in Spoken Interaction.
367994,conf/nips/2016tsw,2019-05-29,5,Proceedings of the NIPS 2016 Time Series Workshop. co-located with the 30th Annual Conference on Neural Information Processing Systems (NIPS 2016). Barcelona. Spain. December 9. 2016


In [8]:
publication=pd.DataFrame.drop_duplicates(publication) # suppression des doublons

In [9]:
publication[publication['article_title'].duplicated(keep=False)] # doublons aussi sur les colonnes !

# Note : certaines publications de mettre titre ont deux id_publication qui différent d'une lettre et certaines ont deux dates différentes 
# pour un même id_publication

,id_publication,date_pub,nbr_authors,article_title
98,conf/3dim/KimMQ18,2019-07-23,3,Multi-planar Monocular Reconstruction of Manhattan Indoor Scenes.
252,conf/3dor/LangenfeldACCDD18,2019-01-29,23,Protein Shape Retrieval.
275,conf/3dor/SongCCHKLMQSTL17,2017-11-24,11,Protein Shape Retrieval.
572,conf/3pgcic/SabaKIPAAJ17,2019-01-08,7,Home Energy Management Using Social Spider and Bacterial Foraging Algorithm.
997,conf/ACISicis/HaqbeenIAO18,2019-10-19,4,Traffic Adaptive Hybrid MAC with QoS Driven Energy Efficiency for WSNs Through Joint Dynamic Scheduling Mode.
...,...,...,...,...
469814,conf/www/VrandecicL19,2019-05-16,2,The First International Workshop on Knowledge Graph Technology and Applications.
469871,conf/www/Wardle19,2019-05-16,1,Enlisting the Public to Build a Healthier Web Information Commons.
469904,conf/www/WuY19,2019-05-16,2,Deep Chit-Chat: Deep Learning for Chatbots.
469994,conf/www/Zhang17b,2018-11-06,1,Smart Jump: Automated Navigation Suggestion for Videos in MOOCs.


In [10]:
delete_rows = publication[publication['article_title'].duplicated(keep='first')] # suppression des doublons sur les colonnes 
publication.drop(delete_rows.index,inplace=True)

In [11]:
publication = publication.loc[np.random.permutation(publication.index)].reset_index(drop=True) # On mélange le dataset, servira par la suite pour d'éventuels tests

In [12]:
publication.head() # dataset mélangé

,id_publication,date_pub,nbr_authors,article_title
0,conf/ccs/0001HPZ17,2019-08-31,4,walk2friends: Inferring Social Links from Mobility Profiles.
1,conf/mobisys/LiuLLW19,2019-10-19,4,Real-time Arm Skeleton Tracking and Gesture Inference Tolerant to Missing Wearable Sensors.
2,conf/icarcv/LlopartRAK18,2019-10-19,4,Online Semantic Segmentation and Manipulation of Objects in Task Intelligence for Service Robots.
3,conf/icinfa/Liu17,2020-03-06,1,Computational awareness for learning neural network ensembles.
4,conf/gi/MunteanuSSW19,2019-11-14,4,On Coresets for Logistic Regression.


In [13]:
len(publication) # nombre d'articles

469611

In [14]:
article_titles = publication["article_title"].str.lower() # conversion des articles en minuscle
article_titles = article_titles.tolist() # conversion des articles en liste

list_id_publication = list(publication['id_publication']) # liste des identifiants

article_titles = dict(zip(list_id_publication,article_titles)) # articles 

In [15]:
#save_dict(article_titles, "original_titles")

#### <span style="color:red">1er nettoyage : suppression des titres non-anglais, des balises et des caractères indésirables et autres aberrations</span>

In [16]:
import string
import re

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [140]:
article_titles = load_dict("original_titles")

In [141]:
article_titles["conf/rocling/WuTHKC17"] # ok 

'?????????????????????? (on the use of sequence labeling and matching methods for asr error detection and correction) [in chinese].'

In [142]:
# le premier nettoyage consiste à supprimer les caractères et les balises qui ne devraient pas se trouver dans les titres des publications (erreurs d'exportation des données)

# Etant donné la diversité des erreurs une perte d'information est possible 
#   -> ex: on veut supprimer tout ce qui se trouve des parenthèses : on ne sait pas ce qu'il y aura
#   dans la parenthèses. La plus part du temps ce sont des abréviations mais ce peut être un mot clef (bien que rare).. Il faut faire des compromis.

for i in article_titles:
    article_titles[i]=re.sub('\[.*?\]',' ',article_titles[i]) # suppression de toutes les expressions entre crochets
    article_titles[i]=re.sub('\{.*?\}',' ',article_titles[i]) # suppression de toutes les expressions entre accolades
    article_titles[i]=re.sub('\(.*?\)',' ',article_titles[i]) # suppression de toutes les expressions entre parenthèses
    article_titles[i]=re.sub('<.+>',' ',article_titles[i])    # suppression de toute la chaine qui se trouve entre un '<' et un '>' (1ère et dernière occurence)
    article_titles[i]=re.sub('\$.+\$',' ',article_titles[i])  # suppression de toute la chaine qui se trouve entre un '$' et un '$' (1ère et dernière occurence)
    article_titles[i]=re.sub('\S*[\^,_,&,:,@,#,$,\\\]\S*',' ',article_titles[i]) # suppression de tous les mots contenant un @ et #.. (@ et # seule inclus aussi)
    article_titles[i]=re.sub('[.,\,,!,?,%,*,§,²,~,\{,\},\(,\)]',' ',article_titles[i]) # suppression des caractères spéciaux
    article_titles[i]=re.sub('[;,+,/]',' ',article_titles[i])
    
# reste à gérer les guillemets simples

In [ ]:
corpus = article_titles

In [103]:
matching = [s for s in article_titles.values() if ')' in s] # test de la présence du mot conference
matching[0:50]

[]

#### <span style="color:orange">Sauvegarde</span>

In [143]:
#save_dict(article_titles, "firstclean")

In [144]:
article_titles = load_dict("firstclean")

#### <span style="color:orange">----------------------------------------------------</span>

#### <span style="color:red"> 2ème nettoyage : Suppression des tirets, des caractères seuls, des stop words et des chaines de caractères référencant des dates ou des pays</span>

In [146]:
article_titles["conf/rocling/WuTHKC17"] # problem

'                           '

In [147]:
for i in article_titles:
    article_titles[i]=re.sub('-',' ',article_titles[i]) # remplacement par un espace, se discute
    article_titles[i]=re.sub(r'\b\d+\b',' ',article_titles[i]) # suppression de tous les chiffres seuls
    article_titles[i]=re.sub('\S*\d[^gd]\S*',' ',article_titles[i]) # suppression de tous les chiffres et des chaines contenant un chiffre excepté les lettres d et g (ne supprime pas les g et d en trop)
    article_titles[i]=re.sub('\S*\d\w{2,}\S*',' ',article_titles[i]) # suppression des éventuels mots avec des plusieurs g ou d à la suite (suite de l'expression d'avant)
    article_titles[i]=re.sub('\S*\w+\d\S*',' ',article_titles[i]) # supression des chaines qui précédent un nombre
    article_titles[i]=re.sub('(\'s)|(\')',' ',article_titles[i]) # suppression des 's ou guillemets simple
    article_titles[i]=re.sub('\S*((january)|(february)|(march)|(april)|(may)|(june)|(july)|(august)|(september)|(october)|(november)|(december))\S*',' ',article_titles[i]) # suppression des expressions qui continnent un mois
    #article_titles[i]=re.sub('([0-9]+)(?:st|nd|rd|th)','',article_titles[i]) # suppression des expression en rapoort avec des dates ou des n-ièmes
    article_titles[i]=re.sub('(conference)|(international)',' ',article_titles[i]) # suppression des mots conference et international qui sont particlièrement fréquents

In [50]:
matching = [s for s in article_titles.values() if '3d' in s] # test de la présence du mot conference
len(matching)

6189

In [148]:
#save_dict(article_titles, "secondclean")

In [149]:
dict(list(article_titles.items())[0:5]) # affichage des 5 premiers éléments du dictionnaire

{'conf/ccs/0001HPZ17': '  inferring social links from mobility profiles ',
 'conf/mobisys/LiuLLW19': 'real time arm skeleton tracking and gesture inference tolerant to missing wearable sensors ',
 'conf/icarcv/LlopartRAK18': 'online semantic segmentation and manipulation of objects in task intelligence for service robots ',
 'conf/icinfa/Liu17': 'computational awareness for learning neural network ensembles ',
 'conf/gi/MunteanuSSW19': 'on coresets for logistic regression '}

In [150]:
article_titles = load_dict("secondclean")

In [151]:
# on veut supprimer les références à des pays et ce qui reste après nettoyage

from country_list import countries_for_language # pip install country_list

countries = dict(countries_for_language('en'))
country_exclusion = [x.lower() for x in list(countries.values())]
country_exclusion = '|'.join(country_exclusion)

for i in article_titles.copy():
    article_titles[i]=re.sub(country_exclusion,' ',article_titles[i])
    article_titles[i]=re.sub(r'\b\D{1,2}\b',' ',article_titles[i]) # On supprime les mots de moins de 3 lettres qui ne contiennent pas de chiffres qui seraient resté
    article_titles[i]=re.sub(' +', ' ', article_titles[i]) # suppression des espaces en trop à la fin du nettoyage
    if (len(article_titles[i])<3) : # suppression des titres à 0 ou 1 mots
        del article_titles[i]

In [153]:
dict(list(article_titles.items())[0:5]) # affichage des 5 premiers éléments du dictionnaire

{'conf/ccs/0001HPZ17': ' inferring social links from mobility profiles ',
 'conf/mobisys/LiuLLW19': 'real time arm skeleton tracking and gesture inference tolerant missing wearable sensors ',
 'conf/icarcv/LlopartRAK18': 'online semantic segmentation and manipulation objects task intelligence for service robots ',
 'conf/icinfa/Liu17': 'computational awareness for learning neural network ensembles ',
 'conf/gi/MunteanuSSW19': ' coresets for logistic regression '}

#### <span style="color:orange">Sauvegarde</span>

In [152]:
#save_dict(article_titles, "nocountries")

In [154]:
article_titles = load_dict("nocountries")

#### <span style="color:orange">----------------------------------------------------</span>

#### <span style="color:red">Suppression des articles qui ne sont pas en anglais</span>

In [133]:
len(article_titles)

469506

In [130]:
for i in article_titles:
    if (article_titles[i].isdigit()==True):
        print(article_titles[i])

In [157]:
for i in article_titles.copy(): # test de la présence dans la chaine de l'unique présence de characteres spéciaux
    # car cela peut causer une erreur dans le test de détection de la langue
    if re.match(r'^[_\W]+$',article_titles[i]):
        print(i)

In [158]:
### Cette celulle prend beaucoup de temps ###   

# On voudrait également supprimer les publications qui ne sont pas en langue anglaise

from langdetect import detect_langs # pip install langdetect

en_ratio = 0.5
for k,v in article_titles.copy().items() :
    res = detect_langs(v)
    # Si le titre n'a aucune trace d'anglais
    if not [l for l in res if l.lang == 'en'] :
        del article_titles[k]
    else :
        for lang in res :
            # Si le titre a des traces d'anglais mais pas suffisamment
            if lang.lang == 'en' and lang.prob < en_ratio :
                del article_titles[k]


In [159]:
len(article_titles)# supression de 30 000 articles

439506

In [162]:
dict(list(article_titles.items())[0:5]) # affichage des 5 premiers éléments du dictionnaire

{'conf/ccs/0001HPZ17': ' inferring social links from mobility profiles ',
 'conf/mobisys/LiuLLW19': 'real time arm skeleton tracking and gesture inference tolerant missing wearable sensors ',
 'conf/icarcv/LlopartRAK18': 'online semantic segmentation and manipulation objects task intelligence for service robots ',
 'conf/icinfa/Liu17': 'computational awareness for learning neural network ensembles ',
 'conf/gi/MunteanuSSW19': ' coresets for logistic regression '}

In [170]:
matching = [s for s in article_titles.values() if 'data science' in s] # test de la présence du mot conference

#### <span style="color:orange">Sauvegarde</span>

In [160]:
#save_dict(article_titles, "onlyenglish")

In [176]:
article_titles = load_dict("onlyenglish") # Les titres des publications sont ici les plus probables

#### <span style="color:orange">----------------------------------------------------</span>

#### <span style="color:red">Suppression des stop words</span>

In [177]:
# tokenisation des titres

tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenized_titles = defaultdict(set)
for k,v in article_titles.items() :
    article_titles[k] = tokenizer.tokenize(v)

#### <span style="color:orange">Sauvegarde</span>

In [178]:
#save_dict(article_titles, "tokenized")

In [179]:
article_titles = load_dict("tokenized")

#### <span style="color:orange">----------------------------------------------------</span>

In [180]:
# suppression des mots vides
import gensim
from gensim.parsing.preprocessing import STOPWORDS # pip install -U gensim

In [181]:
# la suppression des mots vides est différente selon les différentes librairy. Il est néanmoins possible de modifier cette liste.

stopwords = set()
stopwords.update(tuple(nltk.corpus.stopwords.words('english')))
all_stopwords = STOPWORDS.union(stopwords)
print(all_stopwords)

frozenset({'towards', 'sixty', 'hereafter', 'fire', 'wherever', 'beyond', 'she', 'around', 'find', 'moreover', 'the', 'haven', 'that', 'bottom', 'some', 'beforehand', 'became', 'behind', 'or', 'anywhere', "shan't", "aren't", 'someone', 'isn', "mustn't", 'put', 'whatever', 'nine', 'your', 'none', 'fifty', 'enough', "hasn't", 'wasn', 'where', 'shouldn', 'too', 'are', 'needn', 'rather', 'which', 'nowhere', 'elsewhere', 'make', 'else', 'nothing', 'amoungst', 'm', 'several', 'latter', 'part', 've', 'd', "you've", "she's", 'one', 'there', 'mustn', 'after', 'afterwards', 'herein', 'must', 'couldnt', 'de', 'and', 'often', 'become', 'fill', 't', 'between', 'hundred', 'fifteen', 'see', 'once', 'forty', 'computer', 'mostly', 'who', 'didn', 'don', 'go', 'wouldn', 'as', 'seemed', 'what', 'same', 'back', 'had', 'out', 'y', 'whither', 'doesn', 'yet', 'please', 'take', 'system', 'regarding', 'he', 'therein', 'made', 'though', "couldn't", 'latterly', 'last', 'give', 'formerly', 'his', 'everywhere', "it

In [182]:
for k,v in article_titles.items() :
    article_titles[k] = [word for word in v if word not in all_stopwords]

In [184]:
dict(list(article_titles.items())[0:5]) # affichage des 5 premiers éléments du dictionnaire

{'conf/ccs/0001HPZ17': ['inferring',
  'social',
  'links',
  'mobility',
  'profiles'],
 'conf/mobisys/LiuLLW19': ['real',
  'time',
  'arm',
  'skeleton',
  'tracking',
  'gesture',
  'inference',
  'tolerant',
  'missing',
  'wearable',
  'sensors'],
 'conf/icarcv/LlopartRAK18': ['online',
  'semantic',
  'segmentation',
  'manipulation',
  'objects',
  'task',
  'intelligence',
  'service',
  'robots'],
 'conf/icinfa/Liu17': ['computational',
  'awareness',
  'learning',
  'neural',
  'network',
  'ensembles'],
 'conf/gi/MunteanuSSW19': ['coresets', 'logistic', 'regression']}

In [188]:
matching = [s for s in article_titles.values() if 'the' in s] # test de la présence du mot conference
matching

[]

#### <span style="color:orange">Sauvegarde</span>

In [185]:
#save_dict(article_titles, "nostopwords")

In [ ]:
article_titles = load_dict("nostopwords")

#### <span style="color:orange">----------------------------------------------------</span>

#### <span style="color:red">Lemmatisation</span>

In [ ]:
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Romain\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [189]:
# Lemmatisation
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Fonction qui va servir à identité la nature du mot (verbe, nom, ...)
def get_wordnet_pos(word) :
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tags = {'J' : wordnet.ADJ,
            'N' : wordnet.NOUN,
            'V' : wordnet.VERB,
            'R' : wordnet.ADV}
    return tags.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
for k,v in article_titles.items() :
    article_titles[k] = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in v]

In [195]:
dict(list(article_titles.items())[0:5]) # affichage des 5 premiers éléments du dictionnaire

{'conf/ccs/0001HPZ17': ['infer', 'social', 'link', 'mobility', 'profile'],
 'conf/mobisys/LiuLLW19': ['real',
  'time',
  'arm',
  'skeleton',
  'track',
  'gesture',
  'inference',
  'tolerant',
  'miss',
  'wearable',
  'sensor'],
 'conf/icarcv/LlopartRAK18': ['online',
  'semantic',
  'segmentation',
  'manipulation',
  'object',
  'task',
  'intelligence',
  'service',
  'robot'],
 'conf/icinfa/Liu17': ['computational',
  'awareness',
  'learn',
  'neural',
  'network',
  'ensemble'],
 'conf/gi/MunteanuSSW19': ['coresets', 'logistic', 'regression']}

#### <span style="color:orange">Sauvegarde</span>

In [190]:
#save_dict(article_titles, "lemmatized")

In [201]:
article_titles = load_dict("lemmatized")

#### <span style="color:red">Suppression du mot base (fait suite au notebook sur la visualisation)</span>

In [202]:
for i in article_titles:
    for j in article_titles[i].copy(): # on parcours une copie pour éviter certaines erreurs d'index
        if j=="base":
            article_titles[i].remove(j)

In [212]:
dict(list(article_titles.items())[0:5]) # affichage des 5 premiers éléments du dictionnaire

{'conf/ccs/0001HPZ17': ['infer', 'social', 'link', 'mobility', 'profile'],
 'conf/mobisys/LiuLLW19': ['real',
  'time',
  'arm',
  'skeleton',
  'track',
  'gesture',
  'inference',
  'tolerant',
  'miss',
  'wearable',
  'sensor'],
 'conf/icarcv/LlopartRAK18': ['online',
  'semantic',
  'segmentation',
  'manipulation',
  'object',
  'task',
  'intelligence',
  'service',
  'robot'],
 'conf/icinfa/Liu17': ['computational',
  'awareness',
  'learn',
  'neural',
  'network',
  'ensemble'],
 'conf/gi/MunteanuSSW19': ['coresets', 'logistic', 'regression']}

In [213]:
save_dict(article_titles, "cleaned")